In [1]:
import os
import re
from glob import glob
from importlib import reload

import numpy as np

from astropy import units as u
from astropy import coordinates
from astropy.coordinates import SkyCoord
from astropy import table

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

# First parse/load the data 

In [3]:
import data_loaders

In [4]:
elvii_pairs = data_loaders.load_elvii(isolated=False)

Loading /Users/erik/projects/m31-galfa/elvis_data/Burr&Hamilton.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Charybdis&Scylla.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Cher&Sonny.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Hall&Oates.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Kauket&Kek.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Lincoln&Douglas.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Romulus&Remus.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Siegfried&Roy.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Taurus&Orion.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Thelma&Louise.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Venus&Serana.txt
Loading /Users/erik/projects/m31-galfa/elvis_data/Zeus&Hera.txt


In [5]:
#m31 = SkyCoord.from_name('M31')
m31 = SkyCoord(10.6847083*u.deg, 41.26875*u.deg, 700*u.kpc)

In [19]:
res = data_loaders.load_galfa_sensitivity('onempc_mass_sens.fits.zip')
galfa_sens, galfa_sens_scs, galfa_sens_wcs, galfa_sens_hdu = res

/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:672: RuntimeWarning: invalid value encountered in less
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:672: RuntimeWarning: invalid value encountered in greater_equal
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:563: RuntimeWarning: invalid value encountered in less
  if np.any(angles.value < lower) or np.any(angles.value > upper):
/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:563: RuntimeWarning: invalid value encountered in greater
  if np.any(angles.value < lower) or np.any(angles.value > upper):


# Now some LG-simulating behavior

First we produce $M_{HI}$ for everything following Bradford+15

In [7]:
# produce HI masses for each table
for tab in elvii_pairs.values():
    Ms = tab['Mstar_preferred']
    tab['MHI'] = Ms

Now we populate the tables with coordinates for an artificial "LG"

In [54]:
earth_location = [8.5,0, 0]*u.kpc

for tab in elvii_pairs.values():
    #these add host{n}_lat/host{n}_lon/host{n}_dist
    data_loaders.add_oriented_radecs(tab, 0, 1, target_coord=m31, earth_location=earth_location)
    data_loaders.add_oriented_radecs(tab, 1, 0, target_coord=m31, earth_location=earth_location)

/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:736: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if axis == 'z':
/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:740: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif axis == 'y':
/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/coordinates/angles.py:744: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif axis == 'x':


Now we pull the sensitivity from the GALFA image

In [60]:
for tab in elvii_pairs.values():
    for i in (0, 1):
        host_sc = SkyCoord(ra=tab['host{}_lon'.format(i)], 
                           dec=tab['host{}_lat'.format(i)])
        
        pxs = host_sc.to_pixel(galfa_sens_wcs)
        xp = np.round(pxs[0]).astype(int)
        yp = np.round(pxs[1]).astype(int)
        msk = (0<=xp)&(xp<galfa_sens.shape[0])&(0<=yp)&(yp<galfa_sens.shape[1])
        
        sens = np.zeros(len(tab), dtype=galfa_sens.dtype)*galfa_sens.unit
        sens[msk] = galfa_sens[xp[msk], yp[msk]]
        
        tab['closest_sens{}'.format(i)] = sens

And then see if the line is detectable

In [81]:
for tab in elvii_pairs.values():
    MHI = tab['MHI']
    for i in (0, 1):
        sens = tab['closest_sens{}'.format(i)]
        
        MHI_limit = (1*u.Msun*u.s*u.arcmin**-2)/sens#FIX THIS
        
        tab['detectable{}'.format(i)] = MHI_limit > MHI

/Users/erik/miniconda3/envs/anaconda/lib/python3.5/site-packages/astropy/units/quantity.py:822: RuntimeWarning: divide by zero encountered in true_divide
  return super(Quantity, self).__truediv__(other)
